In [1]:
cd E:/GitHub/computervision

E:\GitHub\computervision


In [2]:
import torch
import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

from torch.nn import BCELoss, CrossEntropyLoss
from datetime import datetime
from torch.optim import lr_scheduler, SGD, Adamax
from modules.training_util import PytorchDataset, PytorchTraining, EarlyStopping
from modules.models_util import MobileNet

In [3]:
# Variables for continuing previous training
CONTINUE_TRAINING_DIR = None
# CONTINUE_TRAINING_DIR = 'notebooks/vit/models/'

MODEL_WEIGHTS_PATH = None

if CONTINUE_TRAINING_DIR:
    MODEL_WEIGHTS_PATH = os.path.join(CONTINUE_TRAINING_DIR,'epoch_15.pth')

# CONFIG VARIABLES
FULLY_TRAIN = True
base_path = 'E:/Datasets/cor-splits'
DATA_DIR = [
    # os.path.join(base_path,'sgkf-8-1-1-size-540'),
    # os.path.join(base_path,'underperforming_cameras_96'),
    # os.path.join(base_path,'underperforming_cameras_15'),
    # os.path.join(base_path,'underperforming_cameras_30'),
    os.path.join(base_path,'underperforming_cameras_60_v2'),
    # os.path.join(base_path,'underperforming_cameras_61_15img'),
    # os.path.join(base_path,'underperforming_cameras_59_45imgs'),
    # os.path.join(base_path,'random_cameras_61_30imgs'),
    # os.path.join(base_path,'random_cameras_59_45imgs'),
    # os.path.join(base_path,'random_cameras_71_15img'),
    # os.path.join(base_path,'underperforming_cameras_60_v2_euro'),
    # os.path.join(base_path,'underperforming_cameras_60_v2_itsc'),
    # os.path.join(base_path,'sgkf-8-2-size-1010'),
    # 'E:/Datasets/itsc_flood_dataset'
    ]
START_EPOCH = 1 # Change it from 1 if you're continuing an existing run
N_EPOCHS = 50
BATCH_SIZE = 16
SAVE_INTERVAL = 10 # How many epochs between each backup
## Learning Rate Scheduler
LR = [1e-03,1e-04,1e-05]  # When continuing a existing run, update it to the last epoch LR value
LR_EPOCH_SCHEDULE = 5  # Number of epochs before altering LR
LR_GAMMA = 0.1
## Early Stopping
PATIENCE = 10
MIN_DELTA = 0
MIN_EPOCH = 10 # Number of epochs before starting patience counter
##
BP = [0.5]

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device = {device}")
archs = ['v2']

Device = cuda:0


In [5]:
for arch in archs:
    for dir in DATA_DIR:
        for lr in LR:
            for brightness in BP:
                model_arch = MobileNet(architecture=arch,bp=brightness)
                
                model = model_arch.load(MODEL_WEIGHTS_PATH, FULLY_TRAIN)
                model = model.to(device)
                
                criterion = CrossEntropyLoss()

                if FULLY_TRAIN:
                    params = model.parameters()
                else:
                    params = model.classifier.parameters()
                    
                optimizer = SGD(params, lr=lr)
                # optimizer = Adamax(params, lr=lr)

                step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=LR_EPOCH_SCHEDULE, gamma=LR_GAMMA)
                es = EarlyStopping(patience=PATIENCE, delta=MIN_DELTA, min_epoch=MIN_EPOCH)
                dataset = PytorchDataset(dir, model_arch.data_transforms, BATCH_SIZE)

                working_folder = 'notebooks/mobilenetv2'
                models_folder = 'models'

                curr_time = datetime.now()
                curr_time = curr_time.strftime("%Y-%m-%d %H-%M-%S")

                output_folder = os.path.join(working_folder, models_folder)
                output_folder = os.path.join(output_folder, curr_time)
                
                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)

                trainer = PytorchTraining(device, dataset, output_folder)
                model = trainer.train_pytorch_model(model, criterion, optimizer, step_lr_scheduler, es, START_EPOCH, N_EPOCHS, SAVE_INTERVAL, arch)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to C:\Users\Adm/.cache\torch\hub\checkpoints\mobilenet_v2-7ebf99e0.pth
100.0%


All parameters for model MobileNetV2 requires grad.
Saving model to folder notebooks/mobilenetv2\models\2024-11-18 13-42-33

Model MobileNetV2 v2
Fully Trained = True
Dataset E:/Datasets/cor-splits\underperforming_cameras_60_v2
Learning Rate Epoch Schedule = 5
Learning Rate Gamma = 0.1
Optimizer = SGD
Batch Size = 16


Epoch 1/50
----------

train Loss: 0.6865 Acc: 0.5528
val Loss: 0.6401 Acc: 0.6853
Epoch duration: 0 m 45s
Learning Rate = 0.001



Epoch 2/50
----------

train Loss: 0.6609 Acc: 0.6231
val Loss: 0.6091 Acc: 0.7373
Epoch duration: 0 m 44s
Learning Rate = 0.001



Epoch 3/50
----------

train Loss: 0.6399 Acc: 0.6556
val Loss: 0.5798 Acc: 0.7578
Epoch duration: 0 m 44s
Learning Rate = 0.001



Epoch 4/50
----------

train Loss: 0.6228 Acc: 0.6679
val Loss: 0.5596 Acc: 0.7637
Epoch duration: 0 m 44s
Learning Rate = 0.001



Epoch 5/50
----------

train Loss: 0.6046 Acc: 0.6965
val Loss: 0.5392 Acc: 0.7804
Epoch duration: 0 m 44s
Learning Rate = 0.0001



Epoch 6/50
-------